In [ ]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
# ipython magic function
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier


In [ ]:
# acquire data and get pandas dataframe
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')
combine = [train_df, test_df]

In [ ]:
# analyze by describing data
print(train_df.columns.values)

In [ ]:
# preview the data
#train_df.head(n=2)
train_df.head()

In [ ]:
train_df.tail()

In [ ]:
# pandas dataframe summary
train_df.info()
print('_' * 40)
test_df.info()

In [ ]:
train_df.describe()
# Review survived rate using `percentiles=[.61, .62]` knowing our problem description mentions 38% survival rate.
# Review Parch distribution using `percentiles=[.75, .8]`
# SibSp distribution `[.68, .69]`
# Age and Fare `[.1, .2, .3, .4, .5, .6, .7, .8, .9, .99]`

Pandas [describe function documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.describe.html).

In [ ]:
train_df.describe(include=['O'])

In [ ]:
# include only string columns
train_df.describe(include=[np.object])

In [ ]:
# include only numeric columns
train_df.describe(include=[np.number])